In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pwd
%cd '/content/drive/My Drive/vits'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/vits
/content/drive/My Drive/vits


In [ ]:
!pip install -r requirements.txt
!sudo apt-get install espeak -y
#!pip install unidecode
#!pip install phonemizer
#!pip install pypinyin
#!pip install pyopenjtalk
#!pip install janome
#!pip install jamo
#!pip install cn2an

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.3 MB/s 
     |████████████████████████████████| 183 kB 49.9 MB/s 
     |████████████████████████████████| 11.6 MB 36.5 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 
     |████████████████████████████████| 25.9 MB 1.5 MB/s 
     |████████████████████████████████| 6.8 MB 53.0 MB/s 
     |████████████████████████████████| 748.8 MB 17 kB/s 
     |████████████████████████████████| 5.9 MB 17.0 MB/s 
     |████████████████████████████████| 238 kB 41.2 MB/s 
     |████████████████████████████████| 1.5 MB 50.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 197 kB 52.0 MB/s 
     |████████████████████████████████| 41 kB 609 kB/s 
     |█████████████████████

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 20 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libespeak1 amd64 1.48.04+dfsg-5 [145 

In [ ]:
import os
path = "/content/drive/My Drive/vits"
os.chdir(path)
print(os.getcwd())
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ..

/content/drive/My Drive/vits
/content/drive/My Drive/vits/monotonic_align
running build_ext
/content/drive/My Drive/vits


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm



In [ ]:
#test torch
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.12.1+cu113
False


In [ ]:
umaDict = {'特别周': 0, '无声铃鹿': 1, '东海帝皇': 2, '丸善斯基': 3, '富士奇迹': 4,
           '小栗帽': 5, '黄金船': 6, '伏特加': 7, '大和赤骥': 8, '大树快车': 9, '草上飞': 10,
           '菱亚马逊': 11, '目白麦昆': 12, '神鹰': 13, '好歌剧': 14, '成田白仁': 15, '鲁道夫象征（皇帝）': 16, 
           '气槽': 17, '爱丽数码': 18, '星云天空': 19, '玉藻十字': 20, '美妙姿势': 21, '琵琶晨光': 22, '摩耶重炮': 23, 
           '曼城茶座': 24, '美浦波旁': 25, '目白赖恩': 26, '菱曙': 27, '雪中美人': 28, '米浴': 29, '艾尼斯风神': 30,
           '爱丽速子': 31, '爱慕织姬': 32, '稻荷一': 33, '胜利奖券': 34, '空中神宫': 35, '荣进闪耀': 36, '真机伶': 37,
           '川上公主': 38, '黄金城': 39, '樱花进王': 40, '采珠': 41, '新光风': 42, '东商变革': 43, '超级小海湾': 44,
           '醒目飞鹰': 45, '荒漠英雄': 46, '东瀛佐敦': 47, '中山庆典': 48, '成田大进': 49, '西野花': 50, '春乌拉拉': 51, 
           '青竹回忆': 52, '微光飞驹': 53, '美丽周日': 54, '待兼福来': 55, 'Mr CB': 56, '名将怒涛': 57, '目白多伯': 58,
           '优秀素质': 59, '帝王光辉': 60, '待兼诗歌剧': 61, '生野狄杜斯': 62, '目白善信': 63, '大拓太阳神': 64, 
           '双涡轮': 65, '里见光钻': 66, '北部玄驹': 67, '樱花千代王': 68, '天狼星象征': 69, '目白阿尔丹': 70,
           '八重无敌': 71, '鹤丸刚志': 72, '目白光明': 73, '成田路': 74, '也文摄辉': 75, '小林力奇': 76,
           '北港火山': 77, '奇瑞骏': 78, '苦涩糖霜': 79, '小小蚕茧': 80, '骏川手纲': 81, '秋川弥生': 82,
           '乙名史悦子': 83, '桐生院葵': 84, '安心泽刺刺美': 85, '樫本理子': 86}
configPath = "/content/drive/My Drive/vits/logs/uma.json"
#configPath = "/content/drive/My Drive/vit/logs/config.json"

pthPath = "/content/drive/My Drive/vits/logs/uma466.pth"
#pthPath = "/content/drive/My Drive/vit/logs/365_epochs.pth"

outputPath = "/content/drive/My Drive/vits/output/output.wav"
#读取角色
#角色名
ID=umaDict['东海帝皇']


hps_ms = utils.get_hparams_from_file(configPath)
net_g = SynthesizerTrn(
    len(hps_ms.symbols),
    hps_ms.data.filter_length // 2 + 1,
    hps_ms.train.segment_size // hps_ms.data.hop_length,
    n_speakers=hps_ms.data.n_speakers,
    **hps_ms.model)
print(hps_ms.data.n_speakers)
_ = net_g.eval()

_ = utils.load_checkpoint(pthPath, net_g, None)

outputWav = open(outputPath,'w')

87


In [ ]:
#TODO:实现 说话人|句子的结构 一行一行地读。
words__ = open("/content/drive/My Drive/vits/words.txt",'r',encoding="UTF-8")
words_ = words__.readlines()
words = []
for i in range(len(words_)):
    if words_[i].find("|") != -1:
        words.append(words_[i].split('|'))
print(words)        


[['北部玄驹', ' 「歩夢が部屋に来るの\u3000久しぶりだね」\n'], ['里见光钻', '「そうだね」\n']]


In [ ]:
#stn_tst = get_text("おはよう ", hps_ms)
#with torch.no_grad():
#    x_tst = stn_tst.unsqueeze(0)
#    print(x_tst)
#    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
#    print(x_tst_lengths)
#    sid = torch.LongTensor([2])

#    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid,noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
#ipd.display(ipd.Audio(audio, rate=hps_ms.data.sampling_rate))

In [ ]:
#hps_ms = utils.get_hparams_from_file(configPath)
#print("长度:"+str(len(hps_ms.symbols)))
#print("人数:"+str(hps_ms.data.n_speakers))
#net_g_ms = SynthesizerTrn(
#    len(hps_ms.symbols),
#    hps_ms.data.filter_length // 2 + 1,
#    hps_ms.train.segment_size // hps_ms.data.hop_length,
#    n_speakers=hps_ms.data.n_speakers,
#    **hps_ms.model)
#_ = net_g_ms.eval()
#_ = utils.load_checkpoint(pthPath, net_g_ms, None)
#stn_tst = get_text("おはよう ", hps_ms)

#with torch.no_grad():
#    x_tst = stn_tst.unsqueeze(0)
#    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
#    sid = torch.LongTensor([ID])
#    audio = net_g_ms.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
#ipd.display(ipd.Audio(audio, rate=hps_ms.data.sampling_rate))    

In [ ]:
#批量生成 CPU推理
!pip install pydub
from pydub import AudioSegment
cnt = 1
txt_str=[]
txt_str.append("[Script Info]")
txt_str.append("; Script generated by Aegisub 3.2.2")
txt_str.append("; http://www.aegisub.org/")
txt_str.append("Title: Default Aegisub file")
txt_str.append("ScriptType: v4.00+")
txt_str.append("WrapStyle: 0")
txt_str.append("ScaledBorderAndShadow: yes")
txt_str.append("YCbCr Matrix: TV.601")
txt_str.append("PlayResX: 1440")
txt_str.append("PlayResY: 1080")
txt_str.append("")
txt_str.append("[Aegisub Project Garbage]")
txt_str.append("Audio File: D:/VITS/整活项目/机器人女战士波旁大战吸血鬼米浴/波旁与吸血鬼米浴.mp4")
txt_str.append("Video File: D:/VITS/整活项目/机器人女战士波旁大战吸血鬼米浴/波旁与吸血鬼米浴.mp4")
txt_str.append("Video AR Mode: 4")
txt_str.append("Video AR Value: 1.333333")
txt_str.append("Video Zoom Percent: 0.250000")
txt_str.append("")
txt_str.append("[V4+ Styles]")
txt_str.append("Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour,OutlineColour,BackColour,Bold,Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment,MarginL, MarginR, MarginV, Encoding")
txt_str.append("Style: Default,MS Gothic,45,&H00FFFFFF,&H000000FF,&H00000000,&H00000000,0,0,0,0,100,100,0,0,1,2,2,2,10,10,10,1\n")
txt_str.append("Style: Default,里见光钻,45,&H00FF0000,&H000000FF,&H00000000,&H00000000,0,0,0,0,100,100,0,0,1,2,2,2,10,10,10,1\n")
txt_str.append("Style: Default,北部玄驹,45,&H0000FF00,&H000000FF,&H00000000,&H00000000,0,0,0,0,100,100,0,0,1,2,2,2,10,10,10,1\n")

txt_str.append("")
txt_str.append("[Events]")
txt_str.append("Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text")
total_duration=0
output_sound=[]
def num2timestr( num ):
   return str(int(int(num)/3600))+':'+str(int(int(num)/60))+':'+str(int(int(num)%60))+'.'+str(int(int(num*100)%100))
for word in words:
    #CPU推理

    stn_tst = get_text(word[1], hps_ms)

    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        sid = torch.LongTensor([umaDict[word[0]]])


        audio_ = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
    audio = ipd.Audio(audio_, rate=hps_ms.data.sampling_rate, normalize=False)
    ipd.display(audio)
    write("./output/output"+str(cnt)+".wav",hps_ms.data.sampling_rate,audio_)
    sound = AudioSegment.from_wav("./output/output"+str(cnt)+".wav")
    duration = sound.duration_seconds  # 音频时长（ms）
    print(total_duration)
    print(duration)
    if output_sound==[]:
      output_sound=sound
    else:
      output_sound = output_sound+sound



    txt_str.append("Dialogue: "+num2timestr(total_duration)+','+(num2timestr(total_duration+duration))+','+str(word[0])+',,0,0,0,,｢'+str(word[1])+"｣")
    total_duration=total_duration+duration
    cnt+=1
with open(rf"subtitle.ass", "w", encoding="utf8") as f:
    f.write("\n".join(txt_str))
output_sound.export("/content/drive/My Drive/vits/output.wav", format="wav")  # 保存文件


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


0
3.053424036281179


3.053424036281179
0.9404081632653061


<_io.BufferedRandom name='/content/drive/My Drive/vits/output.wav'>

In [ ]:
#GPU推理
words_ = open("words.txt",'r',encoding="UTF-8")
words = words_.read()
print(words)
text=words



hps_ms = utils.get_hparams_from_file(configPath)
net_g = SynthesizerTrn(
    len(hps_ms.symbols),
    hps_ms.data.filter_length // 2 + 1,
    hps_ms.train.segment_size // hps_ms.data.hop_length,
    n_speakers=hps_ms.data.n_speakers,
    **hps_ms.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint(pthPath, net_g, None)

stn_tst = get_text(words[1], hps_ms)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([ID]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps_ms.data.sampling_rate, normalize=False))

In [ ]:
#单句 CPU推理
print(words[0][1])
#CPU推理
stn_tst = get_text(words[0][1], hps_ms)
with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    sid = torch.LongTensor([umaDict[words[0][0]]])
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps_ms.data.sampling_rate, normalize=False))
print(audio)
write("./output/output.wav",hps_ms.data.sampling_rate,audio_)